# Лабораторная лабора №2 - Проведение исследований с логистической и линейной регрессией


### Подготовка

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sklearn
import keras
import nltk
import pandas as pd
import numpy as np
import re
import torch

from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.preprocessing import LabelEncoder

In [6]:
df = pd.read_excel("/content/drive/MyDrive/ai_course/dataset.xlsx")
df.to_csv("dataset.csv", index = False)
df

,oid,text,category
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,esport
1,749208109,Пусть это побудет здесь БорьбаВпрямомЭфире How...,esport
2,749208109,Раздача пиздюлей от Мунсунга. HowToFtokenoid Б...,esport
3,749208109,Не знаю как вам но мне стилистика нравится пус...,esport
4,749208109,Скриншоты из новой главы. Тэхунчика показали и...,esport
...,...,...,...
53193,910636962,8 битная буря снова накрыла пикселями автомоби...,autosport
53194,669736851,Ира Сидоркова объясняет как сказалась на ее ма...,autosport
53195,558919241,24 я ракетка мира хорват Марин Чилич обыграл и...,tennis
53196,776944963,Стал известен календарь мужской сборной России...,volleyball


In [7]:
df_reg = pd.read_csv("/content/drive/MyDrive/ai_course/train.csv")
df_reg

,cow_id,milk_yield_kg,feed_energy_eke,feed_crude_protein_g,sugar_protein_ratio,breed,pasture_type,sire_breed,milk_fat_pct,milk_protein_pct,milk_taste_label,age_group
0,488,6005,13.5,1842,0.940,РефлешнСоверинг,Равнинное,Айдиал,3.62,3.073,не вкусно,более_2_лет
1,422,5982,13.8,1722,0.890,РефлешнСоверинг,Холмистое,Айдиал,3.61,3.073,не вкусно,более_2_лет
2,105,5700,14.4,1934,0.885,Вис Бик Айдиал,Равнинное,Соверин,3.57,3.079,вкусно,более_2_лет
3,115,5412,12.1,1924,0.890,Вис Бик Айдиал,Равнинное,Соверин,3.57,3.071,не вкусно,менее_2_лет
4,350,6171,15.3,1966,0.940,РефлешнСоверинг,Равнинное,Соверин,3.73,3.076,вкусно,более_2_лет
...,...,...,...,...,...,...,...,...,...,...,...,...
502,72,5809,13.5,2093,0.895,РефлешнСоверинг,Равнинные,Айдиалл,3.61,3.079,не вкусно,более_2_лет
503,107,5417,13.2,1848,0.890,Вис Бик Айдиал,Холмистое,Соверин,3.57,3.076,вкусно,менее_2_лет
504,271,6597,14.2,2204,0.950,РефлешнСоверинг,Равнинное,Соверин,3.74,3.079,не вкусно,более_2_лет
505,436,6054,15.7,1859,0.940,РефлешнСоверинг,Холмистое,Соверин,3.71,3.080,вкусно,более_2_лет


## 2. Создание бейзлайна и оценка качества

### Бейзлайн классификация

In [8]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53198 entries, 0 to 53197
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   oid       53198 non-null  int64 
 1   text      53198 non-null  object
 2   category  53198 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB


In [10]:
RANDOM_STATE = 42

TEXT_COL = "text"
LABEL_COL = "category"

In [11]:
df = df.dropna(subset=[TEXT_COL, LABEL_COL]).reset_index(drop=True)
df

,oid,text,category
0,749208109,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,esport
1,749208109,Пусть это побудет здесь БорьбаВпрямомЭфире How...,esport
2,749208109,Раздача пиздюлей от Мунсунга. HowToFtokenoid Б...,esport
3,749208109,Не знаю как вам но мне стилистика нравится пус...,esport
4,749208109,Скриншоты из новой главы. Тэхунчика показали и...,esport
...,...,...,...
53193,910636962,8 битная буря снова накрыла пикселями автомоби...,autosport
53194,669736851,Ира Сидоркова объясняет как сказалась на ее ма...,autosport
53195,558919241,24 я ракетка мира хорват Марин Чилич обыграл и...,tennis
53196,776944963,Стал известен календарь мужской сборной России...,volleyball


In [ ]:
X_train_balanced_copy = np.copy(X_train_balanced)
y_train_balanced_copy = np.copy(y_train_balanced)
y_test_copy = np.copy(y_test)

In [12]:
le = LabelEncoder()
y = le.fit_transform(df[LABEL_COL].astype(str))
X = df[TEXT_COL].astype(str)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

In [14]:
classification = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95,
        max_features=80_000,
        sublinear_tf=True,
        norm="l2"
    )),
    ("LogisticRegression", LogisticRegression(max_iter=1000, random_state=42))
])

In [15]:
classification.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.95, max_features=80000, min_df=2,
                                 ngram_range=(1, 2), sublinear_tf=True)),
                ('LogisticRegression',
                 LogisticRegression(max_iter=1000, random_state=42))])

In [16]:
y_pred = classification.predict(X_test)

In [17]:
print(classification_report(y_test, y_pred, target_names=le.classes_))

              precision    recall  f1-score   support

   athletics       0.89      0.88      0.88       957
   autosport       0.94      0.81      0.87       630
  basketball       0.92      0.86      0.89       866
  boardgames       0.92      0.95      0.93       910
      esport       0.71      0.87      0.78       902
     extreme       0.66      0.68      0.67       634
    football       0.78      0.79      0.79       852
      hockey       0.96      0.61      0.74       370
martial_arts       0.79      0.80      0.80       852
   motosport       0.90      0.92      0.91       906
      tennis       0.97      0.93      0.95       917
  volleyball       0.88      0.84      0.86       937
winter_sport       0.83      0.89      0.86       907

    accuracy                           0.85     10640
   macro avg       0.86      0.83      0.84     10640
weighted avg       0.86      0.85      0.85     10640



In [18]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=le.classes_, columns=le.classes_)
cm_df

,athletics,autosport,basketball,boardgames,esport,extreme,football,hockey,martial_arts,motosport,tennis,volleyball,winter_sport
athletics,839,0,5,8,25,23,5,2,20,5,0,4,21
autosport,7,510,3,6,27,24,7,0,12,21,0,3,10
basketball,7,1,744,10,35,12,21,1,13,2,2,15,3
boardgames,3,5,1,866,7,8,2,0,6,1,1,5,5
esport,6,2,6,18,785,19,27,1,17,8,2,3,8
extreme,20,4,1,14,57,429,12,0,43,27,0,13,14
football,6,4,16,5,47,16,677,4,25,10,3,21,18
hockey,2,2,10,4,19,10,40,224,6,3,3,18,29
martial_arts,22,3,7,3,41,36,16,0,685,7,3,5,24
motosport,7,5,2,0,16,21,6,0,8,830,0,2,9


#### Выводы по бейзлайну

Результат Weighted F1 = 0.85

Очень хороший результат - модель уверенно отличает большинство классов
Теннис, boardgames имеют очень высокие метрики

hockey по-прежнему тяжело различим

### Бейзлайн регрессия

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [20]:
train = pd.read_csv("/content/drive/MyDrive/ai_course/train.csv")
test = pd.read_csv("/content/drive/MyDrive/ai_course/test.csv")
train.head()

,cow_id,milk_yield_kg,feed_energy_eke,feed_crude_protein_g,sugar_protein_ratio,breed,pasture_type,sire_breed,milk_fat_pct,milk_protein_pct,milk_taste_label,age_group
0,488,6005,13.5,1842,0.940,РефлешнСоверинг,Равнинное,Айдиал,3.62,3.073,не вкусно,более_2_лет
1,422,5982,13.8,1722,0.890,РефлешнСоверинг,Холмистое,Айдиал,3.61,3.073,не вкусно,более_2_лет
2,105,5700,14.4,1934,0.885,Вис Бик Айдиал,Равнинное,Соверин,3.57,3.079,вкусно,более_2_лет
3,115,5412,12.1,1924,0.890,Вис Бик Айдиал,Равнинное,Соверин,3.57,3.071,не вкусно,менее_2_лет
4,350,6171,15.3,1966,0.940,РефлешнСоверинг,Равнинное,Соверин,3.73,3.076,вкусно,более_2_лет


In [21]:
TARGET_COL = "milk_yield_kg"
ID_COL_CANDIDATES = ["cow_id"]

In [22]:
feature_cols = [c for c in train.columns if c != TARGET_COL]

id_col = next((c for c in ID_COL_CANDIDATES if c in test.columns), None)
if id_col is None:
    test_id = pd.Series(test.index, name="row_id")
else:
    test_id = test[id_col].copy()

In [23]:
num_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(train[c])]
cat_cols = [c for c in feature_cols if c not in num_cols]

Минимальные препроцессинг данных для работы модели

In [24]:
num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols)
    ],
    remainder="drop"
)

In [26]:
from sklearn.linear_model import LinearRegression
reg = Pipeline(steps=[
    ("prep", preprocess),
    ("LinearRegression", LinearRegression())
])

In [27]:
X = train[feature_cols]
y = train[TARGET_COL].astype(float)

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

In [29]:
reg.fit(X_tr, y_tr)
y_hat = reg.predict(X_te)

mae  = mean_absolute_error(y_te, y_hat)
mse = mean_squared_error(y_te, y_hat)
r2 = r2_score(y_te, y_hat)
print(f"MAE : {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R^2 : {r2:.4f}")

MAE : 483.9797
MSE: 439490.0998
R^2 : -0.6498


#### Выводы по регрессии

Для задачи прогнозирования удоя коров (milk_yield_kg) была обучена базовая модель LinearRegression из sklearn на исходных числовых и категориальных признаках

MAE = ~ 483 - наша основная метрика, показывает плохой результат

R^2 отрицательное, показывает, что модель предсказывает хуже, чем простое усреднение значений

То есть базовая лин регрессия не справляется с задачей


## 3. Улучшение бейзлайна

### Классификация

Я не буду опять заниматься предопработкой данных, так как вся проделанная работа есть в ЛР 1

Там я скачала итоговые данные и тут буду загружать их и работать с ними далее

Для ЛР 3-5 аналогично

Ожидание:
После балансировки классов и подбора параметров F1-score должен увеличиться, особенно для редких категорий, а общая точность модели станет более стабильной

#### Загрузка данных

In [29]:
import numpy as np
from joblib import load

X_train_embeddings = np.load("/content/drive/MyDrive/ai_course/X_train_embeddings.npy")
X_test_embeddings  = np.load("/content/drive/MyDrive/ai_course/X_test_embeddings.npy")
y_train = np.load("/content/drive/MyDrive/ai_course/y_train.npy")
y_test = np.load("/content/drive/MyDrive/ai_course/y_test.npy")

#### Обучение модели

- Сделаю SMOTE
- Использую GridSearch для оптимизации параметров
- за основную метрику возьму f1_macro, т.к. она одинаково учитывает все классы, даже редкие

Ожидаю, что очистка и эмбеддинги сильно улучшат качество за счет смысловой информации, балансировка должна сильно помочь + оптимизация

In [4]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd

RANDOM_STATE = 42

In [5]:
pipe_logreg = ImbPipeline(steps=[
    ("smote", SMOTE(random_state=RANDOM_STATE)),
    ("norm", Normalizer(norm="l2")),
    ("clf", LogisticRegression(max_iter=5000, random_state=RANDOM_STATE, n_jobs=-1))
])

In [35]:
param_grid = [{
  "clf__solver": ["lbfgs", "saga", "liblinear"],
  "clf__penalty": ["l2"],
  "clf__C": [0.1, 1.0, 10.0],
  "clf__class_weight": [None],
  "clf__multi_class": ["auto"],
}]

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

In [36]:
search_cls = GridSearchCV(
    pipe_logreg,
    param_grid=param_grid,
    scoring="f1_macro",
    cv=cv,
    n_jobs=-1,
    verbose=0,
)

search_cls.fit(X_train_embeddings, y_train)

print("Best params:", search_cls.best_params_)
print("Best f1_macro:", round(search_cls.best_score_, 4))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Best params: {'clf__C': 10.0, 'clf__class_weight': None, 'clf__multi_class': 'auto', 'clf__penalty': 'l2', 'clf__solver': 'saga'}
Best f1_macro: 0.8317


In [37]:
best_logreg = search_cls.best_estimator_
y_pred = best_logreg.predict(X_test_embeddings)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       311
           1       0.86      0.81      0.83       302
           2       0.84      0.79      0.82       307
           3       0.91      0.94      0.92       305
           4       0.79      0.83      0.81       314
           5       0.69      0.79      0.74       306
           6       0.88      0.76      0.81       307
           7       0.79      0.83      0.81       301
           8       0.88      0.85      0.86       313
           9       0.91      0.83      0.87       303
          10       0.93      0.93      0.93       317
          11       0.83      0.85      0.84       316
          12       0.84      0.88      0.86       305

    accuracy                           0.84      4007
   macro avg       0.85      0.84      0.84      4007
weighted avg       0.85      0.84      0.84      4007



#### ИТОГ 3 пункта

Метрики на бейзлайне
```
    accuracy                           0.85     10640
   macro avg       0.86      0.83      0.84     10640
weighted avg       0.86      0.85      0.85     10640
```

Увеличилась метрика fi и accuracy по сравнению с бейзлайном. Большую роль сыграл подбор гипермараметров (тыкала разными варианты, останавилась на такой выборке)

### Регрессия

Базовая линейная регрессия показала слабый результат

Улучшим её за счёт:

- Регуляризации — добавим штраф за большие коэффициенты через Ridge (L2) и Lasso (L1) регрессию

- Подбора гиперпараметра регуляризации

- Предобработка данных

Ожидание:
модель станет устойчивее и точнее предскажет удой

#### Загрузка чистых данных

In [39]:
X_test = pd.read_csv("/content/drive/MyDrive/ai_course/X_test.csv")
X_train = pd.read_csv("/content/drive/MyDrive/ai_course/X_train.csv")
y_test = pd.read_csv("/content/drive/MyDrive/ai_course/y_test.csv")
y_train = pd.read_csv("/content/drive/MyDrive/ai_course/y_train.csv")

#### Обучение

In [13]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [14]:
numeric_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = [c for c in X_train.columns if c not in numeric_features]

In [15]:
num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])
cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore")),
])

prep = ColumnTransformer([
    ("num", num_pipe, numeric_features),
    ("cat", cat_pipe, categorical_features),
])

In [16]:
pipe_reg = Pipeline([
    ("prep", prep),
    ("reg", TransformedTargetRegressor(
        regressor=Ridge(random_state=RANDOM_STATE),
        func=np.log1p, inverse_func=np.expm1
    ))
])

In [17]:
param_grid = [
    {   # Ridge
        "reg__regressor": [Ridge(random_state=RANDOM_STATE)],
        "reg__regressor__alpha": [0.1, 1.0, 10.0],
    },
    {   # Elastic
        "reg__regressor": [ElasticNet(random_state=RANDOM_STATE, max_iter=5000)],
        "reg__regressor__alpha": [0.01, 0.1, 1.0],
        "reg__regressor__l1_ratio": [0.2, 0.5, 0.8],
    },
    {   # Lasso
        "reg__regressor": [Lasso(random_state=RANDOM_STATE, max_iter=5000, selection="random")],
        "reg__regressor__alpha": [0.01, 0.1, 1.0],
    },
]
cv = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

In [18]:
search_reg = GridSearchCV(
    pipe_reg,
    param_grid=param_grid,
    scoring="neg_mean_absolute_error",
    cv=cv,
    n_jobs=-1,
    verbose=1,
)

search_reg.fit(X_train, y_train)
print("Best params:", search_reg.best_params_)
print("Best MAE:", -search_reg.best_score_)

Fitting 3 folds for each of 15 candidates, totalling 45 fits
Best params: {'reg__regressor': Ridge(random_state=42), 'reg__regressor__alpha': 10.0}
Best MAE: 150.95475187537735


In [19]:
best_lin = search_reg.best_estimator_
y_pred = best_lin.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MAE={mae:.3f}  MSE={mse:.3f}  R2={r2:.3f}")


MAE=158.903  MSE=40996.746  R2=0.827


#### ИТОГ РЕГРЕССИЯ


Напоминаю старые метрики

```
MAE : 483.9797
MSE: 439490.0998
R^2 : -0.6498
```

Результат дал меньшую ошибку и r2 положительный. Это связано с тем, что подобраны хорошо параметры и выполнена предобработка. Регуляризация позволила уменьшить переобучение, особенно учитывая, что у нас небольшая выборка. Модель существенно лучше!


## 4. Имплементация алгоритма машинного обучения

### Классификация

Гипотеза: Своя логистическая регрессия (softmax) с L2-регуляризацией и mini-batch SGD при тех же признаках даст качество, сопоставимое со sklearn

In [30]:
import numpy as np
from typing import Optional

def _softmax(z):
    z = z - z.max(axis=1, keepdims=True)
    e = np.exp(z)
    return e / e.sum(axis=1, keepdims=True)

def _one_hot(y, n_classes):
    Y = np.zeros((y.shape[0], n_classes), dtype=np.float32)
    Y[np.arange(y.shape[0]), y] = 1.0
    return Y

In [31]:
class LogisticRegressionNP:
    def __init__(self, lr=0.1, epochs=50, batch_size=1024, l2=1e-4, random_state=42,
                 class_weight: Optional[dict]=None):
        self.lr = lr
        self.epochs = epochs
        self.bs = batch_size
        self.l2 = l2
        self.rng = np.random.RandomState(random_state)
        self.class_weight = class_weight

    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X, dtype=np.float32)
        y = np.asarray(y, dtype=np.int64)
        n, d = X.shape
        self.classes_ = np.unique(y)
        k = self.classes_.size

        inv = {c:i for i,c in enumerate(self.classes_)}
        y_idx = np.array([inv[v] for v in y], dtype=np.int64)

        self.W_ = self.rng.normal(0, 0.01, size=(d, k)).astype(np.float32)
        self.b_ = np.zeros((1, k), dtype=np.float32)

        cw = np.ones(k, dtype=np.float32)
        if self.class_weight is not None:
            for c, w in self.class_weight.items():
                if c in inv:
                    cw[inv[c]] = float(w)

        for _ in range(self.epochs):
            idx = self.rng.permutation(n)
            for s in range(0, n, self.bs):
                e = min(s + self.bs, n)
                xb = X[idx[s:e]]
                yb = y_idx[idx[s:e]]
                Y = _one_hot(yb, k)

                logits = xb @ self.W_ + self.b_
                P = _softmax(logits)

                Wcls = cw[yb][:, None]
                grad_logits = (P - Y) * Wcls

                dW = xb.T @ grad_logits / xb.shape[0] + self.l2 * self.W_
                db = grad_logits.mean(axis=0, keepdims=True)

                self.W_ -= self.lr * dW
                self.b_ -= self.lr * db
        return self

    def predict_proba(self, X: np.ndarray):
        X = np.asarray(X, dtype=np.float32)
        P = _softmax(X @ self.W_ + self.b_)
        return P

    def predict(self, X: np.ndarray):
        P = self.predict_proba(X)
        idx = P.argmax(axis=1)
        return self.classes_[idx]

In [32]:
uniq, cnt = np.unique(y_train, return_counts=True)
class_weight = {int(c): float(cnt.max()/n) for c,n in zip(uniq, cnt)}

In [33]:
logreg_np = LogisticRegressionNP(lr=0.1, epochs=60, batch_size=2048, l2=1e-4,
                                 class_weight=class_weight, random_state=42)

In [34]:
from sklearn.metrics import classification_report

logreg_np.fit(X_train_embeddings, y_train)
y_pred_np = logreg_np.predict(X_test_embeddings)
print(classification_report(y_test, y_pred_np))


              precision    recall  f1-score   support

           0       0.76      0.76      0.76       311
           1       0.75      0.61      0.67       302
           2       0.79      0.71      0.75       307
           3       0.85      0.88      0.87       305
           4       0.72      0.75      0.74       314
           5       0.56      0.69      0.62       306
           6       0.81      0.72      0.76       307
           7       0.68      0.73      0.70       301
           8       0.87      0.79      0.83       313
           9       0.68      0.63      0.66       303
          10       0.87      0.89      0.88       317
          11       0.78      0.74      0.76       316
          12       0.66      0.81      0.72       305

    accuracy                           0.75      4007
   macro avg       0.75      0.75      0.75      4007
weighted avg       0.75      0.75      0.75      4007



#### ИТОГ 4 ПУНКТ


Собственная реализация показала результат сильно хочу, скорее всего из-за отсутствия гиперпараметров

### Регрессия

Гипотезы: своя линейная регрессия на тех же данных даст похожие на sklearn метрики

In [40]:
class LinearRegressionNP:
    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y, dtype=np.float64).reshape(-1, 1)
        X1 = np.c_[X, np.ones((X.shape[0], 1))]
        self.theta_ = np.linalg.pinv(X1.T @ X1) @ (X1.T @ y)
        return self

    def predict(self, X: np.ndarray):
        X = np.asarray(X, dtype=np.float64)
        X1 = np.c_[X, np.ones((X.shape[0], 1))]
        return (X1 @ self.theta_).ravel()

# L2
class RidgeRegressionNP:
    def __init__(self, alpha=1.0):
        self.alpha = alpha

    def fit(self, X: np.ndarray, y: np.ndarray):
        X = np.asarray(X, dtype=np.float64)
        y = np.asarray(y, dtype=np.float64).reshape(-1, 1)
        n, d = X.shape
        X1 = np.c_[X, np.ones((n, 1))]
        I = np.eye(d + 1); I[-1, -1] = 0.0
        self.theta_ = np.linalg.pinv(X1.T @ X1 + self.alpha * I) @ (X1.T @ y)
        return self

    def predict(self, X: np.ndarray):
        X = np.asarray(X, dtype=np.float64)
        X1 = np.c_[X, np.ones((X.shape[0], 1))]
        return (X1 @ self.theta_).ravel()

In [41]:
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

num_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])
cat_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore")),
])

prep = ColumnTransformer([
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])

Xtr_enc = prep.fit_transform(X_train)
Xte_enc = prep.transform(X_test)

In [42]:
Xtr = Xtr_enc.toarray() if hasattr(Xtr_enc, "toarray") else np.asarray(Xtr_enc)
Xte = Xte_enc.toarray() if hasattr(Xte_enc, "toarray") else np.asarray(Xte_enc)

Обучение своей линейной регрессии

In [43]:
lin_np = LinearRegressionNP().fit(Xtr, y_train)

In [44]:
y_pred_lin = lin_np.predict(Xte)
mae = mean_absolute_error(y_test, y_pred_lin)
mse = mean_squared_error(y_test, y_pred_lin)
r2 = r2_score(y_test, y_pred_lin)
print(f"MAE={mae:.3f}  MSE={mse:.3f}  R2={r2:.3f}")


MAE=159.022  MSE=41034.728  R2=0.827


Обучение своей ridge - ожидаю улучшение

In [45]:
ridge_np = RidgeRegressionNP(alpha=1.0).fit(Xtr, y_train)
y_pred_r = ridge_np.predict(Xte)

In [46]:
mae_r = mean_absolute_error(y_test, y_pred_r)
mse_r = mean_squared_error(y_test, y_pred_r)
r2_r = r2_score(y_test, y_pred_r)
print(f"MAE={mae_r:.3f}  MSE={mse_r:.3f}  R2={r2_r:.3f}")

MAE=159.205  MSE=41108.033  R2=0.827


#### ИТОГ РЕГРЕССИЯ

СОбственная реализация дает очень хороший результат, сопоставимый с 3 пунктом, разница между RidgeRegressionNP и LinearRegressionNP. Ridge чуть лучше.

Если сравнивать с бейзлайном, то ЯВНО видно улучшение метрики